<a href="https://colab.research.google.com/github/himanshuRepo/CoLab_Notebooks/blob/master/Neural_Network_in_Tensorflow_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network on MNIST Dataset

##Mount the Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Check the working directory

In [0]:
!ls "/content/drive/My Drive/"

##Change to a specific folder on Google Drive

In [0]:
import os
os.chdir("drive/My Drive/")

## Displaying the considered neural network

In [0]:
from IPython.display import Image
Image('./images/Page34.jpg')

* Creating a simple three layer neural network by using the
  MNIST dataset that the TensorFlow package provides

* This MNIST dataset is a set of 28 X 28 pixel grayscale images
  which represent hand-written digits

* It has 55,000 training rows, 10,000 testing rows and 5,000
  validation rows

## Importing the libraries

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

# Load Dataset

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [0]:
# Get the first images from the test-set.
images = mnist.test.images[0:9]
plt.imshow(images[0].reshape(28,28), cmap='binary')
plt.axis('off')
plt.show()

# Variable Set up

In [0]:
# Python optimisation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

# Weight and Bias Code

In [0]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

# Hidden Layer Node Setup

In [0]:
# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

# Output Layer Node Setup

Setup the output layer, y_

In [0]:
# now calculate the hidden layer output - in this case, let's use a softmax activated
# output layer
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

# Optimisation function

Cross entropy cost function

In [0]:
from IPython.display import Image
Image('./images/crossentropy.png')

In [0]:
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)+ (1 - y) * tf.log(1 - y_clipped), axis=1))

# Optimizer Algorithm Setup

This function will then perform the gradient descent and the
backpropagation for you.

In [0]:
# add an optimiser
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)


# Accuracy Code

Setup the variable initialisation operation and an operation to
measure the accuracy of our predictions

In [0]:
# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Setup Code for the Training Process

In [0]:
# start the session
with tf.Session() as sess:
  # initialise the variables
  sess.run(init_op)
  total_batch = int(len(mnist.train.labels) / batch_size)
  for epoch in range(epochs):
    avg_cost = 0
    for i in range(total_batch):
      batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
      _, c = sess.run([optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y})
      avg_cost += c / total_batch
      print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
  print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))
  summary_writer = tbc.get_writer()
  summary_writer.add_graph(sess.graph)